In [68]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')


In [69]:
#load signal 
df_signal = pd.read_csv("signal.csv", sep=";")
df_signal.dropna(axis=[1,0], how='all', inplace=True)
print("Signal Features: {}".format(len(df_signal.columns)))

#load background
df_background = pd.read_csv("background.csv", sep=";")
df_background.dropna(axis=[1,0], how='all', inplace=True)
print("Background Features: {}".format(len(df_background.columns)))

#lets only take columns that appear in both datasets
df = pd.concat([df_signal, df_background], axis=0, join='inner')

#match  all columns containing the name 'corsika'
c = df.filter(regex="((C|c)orsika)(\w*[\._\-]\w*)?").columns
df = df.drop(c, axis=1)

#match any column containing utc, mjd, date or ID. Im sure there are better regexes for this.
c = df.filter(regex="\w*[\.\-_]*(((u|U)(t|T)(c|C))|((m|M)(j|J)(d|D))|((d|D)ate)|(ID))+\w*[\.\-_]*\w*").columns
df = df.drop(c, axis=1)

print("Combined Features: {}".format(len(df.columns)))
df.describe()

Signal Features: 264
Background Features: 224
Combined Features: 201


,HitMultiplicityValues.n_hit_strings,HitMultiplicityValues.n_hit_doms,HitMultiplicityValues.n_hit_doms_one_pulse,HitStatisticsValues.cog_z_sigma,HitStatisticsValues.min_pulse_time,HitStatisticsValues.max_pulse_time,HitStatisticsValues.q_max_doms,HitStatisticsValues.z_min,HitStatisticsValues.z_max,HitStatisticsValues.z_mean,...,Weight.Astro2,NewAtt.radius,NewAtt.DirectEllipse,NewAtt.DeltaZd,NewAtt.AbsSmooth,NewAtt.emptyness,NewAtt.SepDevide,NewAtt.SPEBayVerRadius,NewAtt.SplineVerRadius,Weight.HoSa
count,40000.00000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,40000.000000,...,4.000000e+04,40000.000000,40000.000000,40000.000000,39639.000000,40000,40000.000000,40000.000000,40000.000000,4.000000e+04
mean,8.66845,27.702150,18.050100,66.156127,9925.463629,12406.666852,15.768365,-106.537761,151.562618,14.778464,...,1.275894e+00,355.021280,58.902182,0.091101,0.337847,1,0.718036,402.749254,363.301351,1.279224e+00
std,5.08321,26.269488,14.902746,53.125887,941.970049,1435.253836,79.346853,357.235969,319.998073,340.730131,...,2.139744e+00,138.637692,65.109523,0.126985,0.146087,0,0.308382,191.932780,138.326494,2.137763e+00
min,1.00000,8.000000,1.000000,8.796791,8277.723633,10027.590820,1.100601,-512.820007,-455.559998,-490.288333,...,1.110806e-11,0.777723,11.163259,0.000001,0.000013,1,-1.300890,0.000000,2.419013,1.110806e-11
25%,5.00000,15.000000,10.000000,31.619315,9864.070557,11630.747314,3.554974,-494.309998,-169.759995,-334.261756,...,1.022837e-06,262.076799,18.128552,0.021336,0.234974,1,0.528290,284.485836,270.543895,4.792810e-03
50%,7.00000,20.000000,14.000000,47.214615,9876.689453,12156.053711,5.817134,-59.099998,229.149994,55.810077,...,1.058097e-04,383.907246,33.219237,0.054961,0.319149,1,0.726513,418.051743,392.165084,1.428358e-02
75%,11.00000,30.000000,21.000000,80.395136,9885.310547,12862.305420,11.371012,231.940002,485.769989,345.590913,...,1.494497e+00,468.332911,68.215974,0.113048,0.426644,1,0.886926,508.052516,474.978558,1.494497e+00
max,57.00000,720.000000,243.000000,449.089998,49808.296875,57779.906250,6506.370362,465.769989,524.559998,494.442999,...,7.635787e+00,600.071165,613.402922,2.344152,0.868363,1,3.658548,2237.181550,630.270158,7.635787e+00
